<a href="https://colab.research.google.com/github/chuducthang77/Deep-Learning-project/blob/main/CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install the required package
!pip install insightface
!pip install onnx
!pip install mxnet-cu101

In [18]:
#Import necessary package
import insightface
import urllib
import urllib.request
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

In [19]:
#Change image to array
def image_to_array(image):
  image_array = np.asarray(image, dtype="uint8")
  #image_array = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
  return image_array

In [53]:
#Read the image and store in the list
def load_image(name):
  path = '/content/drive/MyDrive/Machine Learning/Course work 2/img/' + name 
  img = cv2.imread(path)
  #Process the image to the array-type
  image = image_to_array(img)

  #Resize the image
  image = cv2.resize(image, (112,112))
  
  return image

In [54]:
#Load all the image 
directory = r'/content/drive/MyDrive/Machine Learning/Course work 2/img'
filename = os.listdir(directory)
images = []
for name in filename:
  images.append(load_image(name))

In [22]:
#Use the arcface model
model = insightface.model_zoo.get_model('arcface_r100_v1')

In [23]:
#Prepare for the model to use CPU to recognize images (positive number for GPU)
model.prepare(ctx_id=-1)

In [55]:
#Create the embedding matrix
emb_matrix = model.get_embedding(images[0])
for i in range(1, len(images)):
  emb = model.get_embedding(images[i])
  emb_matrix = np.vstack((emb_matrix, emb))

In [56]:
print(emb_matrix.shape)

(140, 512)


In [67]:
#Calculate the cosine distance
def cosine_distance(train, test):
  numerator = np.sum(np.multiply(train, test))
  denominator = np.sqrt(np.sum(np.square(train))) * np.sqrt(np.sum(np.square(test)))
  return numerator/denominator

In [71]:
#Predict what is the name of the person in the image
def predict(cosine_distance):
  
  name_list = ['Đức', 'HĐức', 'Hiếu', 'Hùng', 'Kiên', 'Linh', 'Quân','Tân','Thắng','Trường','Tuấn','Vân','Việt Đức','Xuân Anh']
  min_distance = min(cosine_distance)
  threshold = 0.2

  if min_distance <= threshold:
    index = cosine_distance.index(min_distance)
    index = int(index/10)
    return name_list[index]
  else:
    return 'unknown'


In [72]:
from scipy import spatial

#Load the test image
test = cv2.imread('/content/drive/MyDrive/Machine Learning/Course work 2/Đức1.jpg')

#Resize the test image
test = image_to_array(test)
test = cv2.resize(test, (112,112))

#Get the embedding for test image
test_emb = model.get_embedding(test)

distance = []
for i in range(emb_matrix.shape[0]):
  distance.append(cosine_distance(emb_matrix[i,:], test_emb))

print(predict(distance))

Đức
